### This script aggregates the nationalID raster and the GDP raster, maintaining national GDP totals

In [1]:
import rasterio
import numpy as np
import dask.array as da
from scipy.stats import mode

In [2]:
# Load the datasets
with rasterio.open('/home/mark/projects/open-gira/results/direct/WP/NatID_1km_test.tif') as src:
    national_id = src.read(1)
    profile = src.profile
# with rasterio.open('/home/mark/projects/open-gira/results/direct/WP/GDP2005_1km.tif') as src:
#     gdp = src.read(1)

In [3]:
# Define the new resolution
factor = 3
block_size = (factor, factor)

In [4]:
# Create a dask array from the national ID data
national_id_da = da.from_array(national_id, chunks=block_size)

In [8]:
# Define function to compute the mode of a block
def block_mode(block):
    return mode(block.ravel()).mode[0]

In [9]:
# Use dask's map_blocks function to apply block_mode to each block of the array
national_id_agg = national_id_da.map_blocks(block_mode, dtype=national_id_da.dtype)

In [10]:
# Compute the result
national_id_agg = national_id_agg.compute()

IndexError: invalid index to scalar variable.

In [ ]:
# Update the profile for the new resolution
profile.update(
    dtype=rasterio.int32,  # or whatever dtype is appropriate for your data
    height=national_id_agg.shape[0],
    width=national_id_agg.shape[1],
    transform=rasterio.Affine(0.1, 0, profile['transform'][2],
                              0, -0.1, profile['transform'][5])
)

In [ ]:
with rasterio.open('/home/mark/projects/open-gira/results/direct/WP/national_id_agg.tif', 'w', **src.profile) as dst:
    dst.write(national_id_agg, 1)